### CS4423 - Networks
Angela Carnevale  
School of Mathematical and Statistical Sciences  
University of Galway

#### 3. Centrality Measures

# Week 6, lecture 1: Closeness and Betweenness Centrality.  Examples.

We conclude our discussion on centrality measures. Today's centrality measures are based on distance and shortest paths. These are: **closeness** and **betweenness** centrality.


In [ ]:
import networkx as nx
import pandas as pd
import yaml
from queue import Queue
opts = { "with_labels": True, "node_color": 'y'}

Next, recover the graph `G` of marital ties between Florentine families, together with the node attributes we have already determined.

In [ ]:
with open('data/florentine.yml', 'r') as f:
    G=yaml.load(f,Loader=yaml.Loader)
    

We stored information on: degree and eigenvector centrality:

In [ ]:
G.nodes['Guadagni']


In [ ]:
pd.DataFrame.from_dict(
    dict(G.nodes(data=True)), 
    orient='index'
).sort_values('degree', ascending=False)

## Closeness Centrality

A node $x$ in a network can be regarded as being central, if it is **close** to (many) other nodes, 
as it can then quickly interact with them. 

As "lower distance &harr; greater importance", a simple way to measure closeness in this sense
is based on the sum of all the distances to the other nodes, as follows.

**Definition (Closeness Centrality).**
In a simple, **connected** graph $G$ of order $n$, the **closeness centrality** $c_i^C$ of node $i$
is defined as
$$
c_i^C = \frac{1}{\sum_{j=1}^n d_{ij}}.
$$

As always, together with the raw numbers, we consider the normalised ones.

The **normalised closeness centrality** of node $i$, defined as
$$
C_i^C = (n-1) c_i^C
$$
takes values in the interval $[0, 1]$.
</div>

**Problem.** How to compute $c_i^C$? 

**BFS again.**  

* The following `python` function implements
BFS for shortest distance from a previous lecture.  
* It takes a graph $G = (X, E)$ and a vertex $x \in X$
as its arguments. 
* It returns a **dictionary**, which assigns to each node its distance to $x$.

In [ ]:
def distances(G, x):
    
    # 1. init: set up the dictionary and a queue
    dists = { y: None for y in G } # distances
    Q = Queue() # queue of nodes to be visited
    dists[x] = 0
    Q.put(x)
    
    # 2. loop
    while not Q.empty(): 
        y = Q.get() 
        for z in G.neighbors(y):
            if dists[z] is None:
                dists[z] = dists[y] + 1
                Q.put(z)
    
    # 3. stop here
    return dists

* For example, the distance from node `'Medici'` to all nodes in `G`:

In [ ]:
d = distances(G, 'Medici')
print(d)
G.order()

**Note.** 

* On the isolated node `'Pucci'` the above would have returned an undefined distance from all nodes other than itself. 

* When a node is isolated, the sum of all the distances from it is $0$, causing a **division-by-zero error**. But we've removed the only isolated node in our graph so we're safe.

* Use `distances` to compute the normalized closeness centrality according to the above
definition.

In [ ]:
n = G.order()
closeness = { x : (n-1)/sum(distances(G, x).values()) for x in G }

In [ ]:
closeness

* Compare the results to the `networkx` version of closeness:

In [ ]:
nx.closeness_centrality(G)

In [ ]:
nx.closeness_centrality(G) == closeness

* Let's add those measurements to the table.

In [ ]:
nx.set_node_attributes(G, closeness, '$C_i^C$')

In [ ]:
pd.DataFrame.from_dict(
    dict(G.nodes(data=True)), 
    orient='index'
).sort_values('degree', ascending=False)

We update our file with the most recent data for later use.

In [ ]:
with open('data/florentine.yml', 'w') as f:
    yaml.dump(G,f)
    

## Betweenness Centrality

Recall the definitions of **betweenness centrality** of a node and its normalised analogue.

**Definition (Betweenness Centrality).**
In a simple, connected graph $G$, the **betweenness centrality** $c_i^B$ of node $i$
is defined as
$$
c_i^B = \sum_{j \neq i} \sum_{k \neq i} \frac{n_{jk}(i)}{n_{jk}},
$$
where $n_{jk}$ denotes the **number** of shortest paths from
node $j$ to node $k$, and where $n_{jk}(i)$ denotes the
number of those shortest paths **passing through** node $i$.

The **normalised betweenness centrality** of node $i$, defined as
$$
C_i^B = \frac{c_i^B}{(n-1)(n-2)}
$$
takes values in the interval $[0, 1]$.
</div>

* Note that $(n-1)(n-2)$ is the largest number of shortest paths beween pairs of distinct nodes that a given node could possibly sit on.

* How to compute $C_i^B$?

**BFS once more.**  This time as follows:

* A python function which returns a **dictionary** that contains, for each node $x$, a list of **immediate predecessors** of $y$ in a shortest path from $x$ to $y$.  

* We have already seen that this another piece of information that BFS can determine
on the fly: when constructing a **spanning tree** while traversing a graph, we need to remember **one**
immediate predecessor for each newly discovered node.  

* Here we determine and remember **all** immediate
predecessors, requiring little if no extra work.

From this list of predecessors, one can then recursively reconstruct **all shortest paths** from $x$ to $y$.
We still need to keep track of the shortest path lengths in order to decide which neighbor $x$
actually is a predecessor of $y$.

In [ ]:
def predecessors(G, x):
    
    # 1. init: set up the two dictionaries and queue
    dists = { y: None for y in G } # distances
    preds = { y: [] for y in G } 
    Q = Queue()
    dists[x] = 0
    Q.put(x)
    
    # 2. loop
    while not Q.empty():
        y = Q.get()
        for z in G.neighbors(y):
            if dists[z] is None:
                dists[z] = dists[y] + 1
                preds[z].append(y)
                Q.put(z)
            elif dists[z] > dists[y]:
                preds[z].append(y)
    
    # 3. stop here
    return preds

In [ ]:
G = nx.Graph()
G.add_edges_from(['ab','ac','ad','ae','bc','bf','cf','eh','dh','fi','ig','dg','hj','gj','ik','jk'])

In [ ]:
predecessors(G,'a')

In [ ]:
p = predecessors(G, 'Medici')
p

In [ ]:
nx.draw(G, **opts)

Using the **predecessor lists** with respect to $x$, the **shortest paths** from $x$ to $y$ can be enumerated recursively:
* if $y = x$: the shortest path from $x$ to itself is the empty path starting an ending at $x$.
* else, if $y \neq x$ then each shortest path from $x$ to $y$ travels through
  exactly one of $y$'s predecessors ... and ends in $y$.

So, in formulas, $S_x(x) = \{(x)\}$ and
$$
S_x(y) = \{ p + (y) : p \in S_x(z),\, z \in \mathrm{pre}_x(y)\}
$$

In [ ]:
def shortest_paths(x, y, pre):
    if x == y:
        return [[x]]
    paths = []
    for z in pre[y]:
        for path in shortest_paths(x, z, pre):
            paths.append(path + [y])
    return paths

In [ ]:
def spaths(x, y, pre):
    if x == y:
        return [[x]]
    return [ p + [y] for p in spaths(x, z, pre) for z in pre[y] ]

In [ ]:
shortest_paths('Medici', 'Bischeri', p)

* Now compute betweenness:

In [ ]:
betweenness = { x : 0.0 for x in G }
n = G.order()

In [ ]:
for x in G: 
    pre = predecessors(G, x)
    for y in G:
        paths = shortest_paths(x, y, pre)
        njk = len(paths)*(n-1)*(n-2)  # normalize
        for p in paths:
            for z in p[1:-1]:  # exclude endpoints
                betweenness[z] += 1/njk # add 1 (normalised) to betweenness of x
                                        # every time a shortest path passes through z

In [ ]:
betweenness

* Compare the results to the `networkx` version of betweenness:

In [ ]:
nx.betweenness_centrality(G)

In [ ]:
nx.draw(G, **opts)

* Finally, let's add the normalized betweenness centralities as attributes to the
nodes of the graph, and display the resulting table.

In [ ]:
nx.set_node_attributes(G, betweenness, '$C_i^B$')

In [ ]:
pd.DataFrame.from_dict(
    dict(G.nodes(data=True)), 
    orient='index'
).sort_values('degree', ascending=False)

In [ ]:
with open('data/florentine.yml', 'w') as f:
    yaml.dump(G,f)

##  Summary and examples

There are many different ways to be important.  As a node in a network, you are important if
* you have **many friends** (degree centrality)
* you have **important friends** (eigenvector centrality)
* you are **close** to many (closeness centrality)
* many interactions **pass through** you (betweenness centralty).

Recall that $C_i^C$ is the normalized closeness centrality of node $i$.  Why
   is $0 \leq C_i^C \leq 1$?  When is $C_i^C = 1$?  Is $C_i^C$ ever $0$?



In a graph of order $n$, the **normalised closeness centrality** of node $i$, defined as
$$
C_i^C = (n-1) c_i^C= {(n-1)}{\left(\sum_{j=1}^n d_{ij}\right)^{-1}}
$$
takes values in the interval $[0, 1]$.

In a connected network of order $n$, a node of degree $n-1$ will have normalised closeness centrality $1$.

Recall that $C_i^B$ is the normalized betweenness centrality of node $i$.
   Why is $0 \leq C_i^B \leq 1$?  When is $C_i^B = 1$?  Is $C_i^B$ ever $0$?

A leaf (node of degree 1) will have betweenness centrality 0.

##  Code Corner

### `networkx`

* `closeness_centrality`: [[doc]](https://networkx.github.io/documentation/stable/reference/algorithms/generated/networkx.algorithms.centrality.closeness_centrality.html#networkx.algorithms.centrality.closeness_centrality)
   
    
* `betweenness_centrality`: [[doc]](https://networkx.github.io/documentation/stable/reference/algorithms/generated/networkx.algorithms.centrality.betweenness_centrality.html#networkx.algorithms.centrality.betweenness_centrality)